In [108]:
import numpy as np
from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation

seq_len = 40
step = 3

In [77]:
file = open('data/text_corpus.txt', 'r')
text = file.read().lower().strip()
to_replace = ['\n', '\t', '\xa0',
 '¡', '¢', '£', '§', '«', '°', '±', '´', '·', '»', '¼', '½', '¾', '¿', '×', 'ß', 'à', 'á', 'â', 'ã', 'ä', 'å', 'æ', 'ç', 'è',
 'é', 'ê', 'ë', 'ì', 'í', 'î', 'ï', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ù', 'ú', 'û', 'ü', 'ÿ', 'š', '–', '—', '‘', '’', '“', '”', 
 '„', '†', '•', '…', '‹', '›','¹', 'õ', 'ž','ð',  '®', '¯', '²', 'µ', 'œ',
             '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', 
             ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`',
             '{', '|', '}', '~', '¥', '‚', '€']
for tr in to_replace:
    text = text.replace(tr, '')


In [78]:
print(text[:1000])
print(len(text))

the exclamation mark british english or exclamation point some dialects of american english is a punctuation mark usually used after an interjection or exclamation to indicate strong feelings or high volume shouting or show emphasis and often marks the end of a sentence example watch out similarly a bare exclamation mark with nothing before or after is often used in warning signsother uses includegraphically the exclamation mark is represented as a full stop point with a vertical line above one theory of its origin is that it is derived from a latin exclamation of joy io the modern graphical representation is believed to have been born in the middle ages medieval copyists wrote the latin word io at the end of a sentence to indicate joy the word io meant hurray over time the i moved above the o and the o became smaller becoming a pointthe exclamation mark was first introduced into english printing in the 15th century to show emphasis and was called the sign of admiration or exclamation 

In [79]:
unique_chars = sorted(list(set(text)))
print(len(unique_chars))
num_chars = len(unique_chars)

37


In [80]:
print(unique_chars, end = ' ')

[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'] 

In [81]:
def one_hot_char(idx, size):
    oh = np.zeros((size))
    oh[idx] = 1
    return oh

In [82]:
def reverse_pred(pred, top_n = 1):
    nth_top = np.sort(pred)[-top_n]
    idx = np.argwhere(pred == nth_top)
    return idx[0][0]

In [83]:
i = 0                                # maybe start with 1
char_to_idx = {}
idx_to_char = {}
for char in unique_chars:
    char_to_idx[char] = i
    idx_to_char[i] = char
    i = i+1

In [84]:
t = one_hot_char(char_to_idx[' '], num_chars)
print(t)
print(char_to_idx[' '])

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
0


In [85]:
def char_model():
    inputs = Input(shape = (seq_len, num_chars))
    X = LSTM(128, return_sequences = True)(inputs)
    X = Dropout(0.5)(X)
    X = LSTM(128)(X)
    X = Dense(num_chars, activation = 'softmax')(X)
    
    model = Model(inputs = inputs, outputs = X)
    return model

In [86]:
X_train = []
Y_train = []

# pad_vector = [0]*num_chars
# pad_vector[0] = 1

for i in range(0, 100000 - seq_len, step):
#     random_len = np.random.randint(0, seq_len)
#     to_pad = seq_len - random_len
#     pad_vector = [pad_vector] * to_pad
#     temp = text[i:i+random_len]
    temp = text[i:i+seq_len]
#     y = one_hot_char(char_to_idx[text[i+random_len]], num_chars)
    y = one_hot_char(char_to_idx[text[i+seq_len]], num_chars)
    x = [one_hot_char(char_to_idx[char], num_chars) for char in temp]
#     print(temp , ' : ', text[i+random_len])
#     print(temp , ' : ', text[i+seq_len])

#     X_train.append([one_hot_char(char_to_idx[char], num_chars) for char in temp] + pad_vector)
    X_train.append(x)
    Y_train.append(y)
X_train = np.array(X_train)
Y_train = np.array(Y_train)

In [87]:
print(X_train.shape)
print(Y_train.shape)

(33320, 40, 37)
(33320, 37)


In [88]:
print(X_train[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [89]:
model = char_model()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [90]:
model.fit(X_train, Y_train, epochs = 10, batch_size = 32, shuffle = True)

Epoch 1/10
33320/33320 [==============================] - 144s 4ms/step - loss: 2.7097 - acc: 0.2231
Epoch 2/10
33320/33320 [==============================] - 143s 4ms/step - loss: 2.3070 - acc: 0.3163
Epoch 3/10
33320/33320 [==============================] - 143s 4ms/step - loss: 2.1471 - acc: 0.3615
Epoch 4/10
33320/33320 [==============================] - 142s 4ms/step - loss: 2.0419 - acc: 0.3925
Epoch 5/10
33320/33320 [==============================] - 142s 4ms/step - loss: 1.9567 - acc: 0.4173
Epoch 6/10
33320/33320 [==============================] - 142s 4ms/step - loss: 1.8863 - acc: 0.4391
Epoch 7/10
33320/33320 [==============================] - 142s 4ms/step - loss: 1.8250 - acc: 0.4566
Epoch 8/10
33320/33320 [==============================] - 142s 4ms/step - loss: 1.7654 - acc: 0.4716
Epoch 9/10
33320/33320 [==============================] - 143s 4ms/step - loss: 1.7143 - acc: 0.4882
Epoch 10/10
33320/33320 [==============================] - 142s 4ms/step - loss: 1.6673 - a

In [96]:
def predict_till_space(sequence, model):
    original = sequence.lower()
    final = []
    for i in range(1, 6):
        output = ""
        sequence = [[one_hot_char(char_to_idx[char], num_chars) for char in original]]
        pred = model.predict(np.array(sequence))[0]
        y = one_hot_char(reverse_pred(pred, top_n = i), num_chars)
        sequence[0].append(y)
        del sequence[0][0]
        y = idx_to_char[reverse_pred(pred, top_n = i)]
        output += y
        while True:
            pred = model.predict(np.array(sequence))[0]
            y = one_hot_char(reverse_pred(pred), num_chars)
            sequence[0].append(y)
            del sequence[0][0]
            y = idx_to_char[reverse_pred(pred)]
            if y == ' ':
                break
            output +=y
        final.append(output)
    return final

In [107]:
test = ['There are other martyrs used by country ',
        'War Thunder is the most comprehensive ga',
        'They thought it was all over the nation ',
        'highly detailed and personalized aviatio']
for phrase in test:
    results = predict_till_space(phrase, model)
    print(phrase, ":", results)

There are other martyrs used by country  : ['be', 'of', 'to', 'some', 'and']
War Thunder is the most comprehensive ga : ['s', 'nd', 've', 'me', 'le']
They thought it was all over the nation  : ['of', 'a', 'in', 'the', 'for']
highly detailed and personalized aviatio : ['n', ' a', 'l', 'r', 'u']
